<a href="https://colab.research.google.com/github/aunfer/hello-world/blob/main/Projeto_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Imports + Definition of databases**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests


url_taxas_juros="https://olinda.bcb.gov.br/olinda/servico/taxaJuros/versao/v2/odata/TaxasJurosMensalPorMes?$top=10000&$format=json&$select=Mes,Modalidade,Posicao,InstituicaoFinanceira,TaxaJurosAoMes,TaxaJurosAoAno,cnpj8,anoMes"
url_expecs="https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/odata/ExpectativasMercadoInflacao12Meses?$format=json&$select=Indicador,Data,Media,Mediana"
url_ipca="https://servicodados.ibge.gov.br/api/v3/agregados/6691/periodos/201701|201702|201703|201704|201705|201706|201707|201708|201709|201710|201711|201712|201801|201802|201803|201804|201805|201806|201807|201808|201809|201810|201811|201812|201901|201902|201903|201904|201905|201906|201907|201908|201909|201910|201911|201912|202001|202002|202003|202004|202005|202006|202007|202008|202009|202010|202011|202012|202101|202102|202103|202104|202105|202106|202107|202108|202109|202110|202111|202112|202201|202202|202203|202204|202205|202206|202207|202208|202209|202210|202211|202212|202301|202302|202303|202304|202305|202306|202307|202308|202309|202310/variaveis/2266|63|9798|9799|9800|9801?localidades=N1[all]"


## **Categorization of API errors**

In [2]:
codigos=pd.DataFrame({("200","OK"),("201","Created"),("203","Non-Authoritative Information"),("204","No Content"),("400","Bad Request"),("401","Unauthorized"),("403","Forbidden"),("404","Not Found"),("409","Conflict"),("500","Internal Server Error"),("503","Service Unavailable"),("501","Not Implemented"),("504","Gateway Timeout"),("599","Network Timeout"),("502","Bad Gateway")}, columns =["Codigo", "Mensagem"])
codigos.set_index("Codigo",inplace=True)
codigos

,Mensagem
Codigo,
501,Not Implemented
404,Not Found
201,Created
400,Bad Request
403,Forbidden
500,Internal Server Error
200,OK
409,Conflict
599,Network Timeout


## **Requests**

In [3]:
response_taxas_juros=requests.get(url_taxas_juros)
response_taxas_juros.status_code

response_expecs=requests.get(url_expecs)
response_expecs.status_code

response_ipca=requests.get(url_ipca)
response_ipca.status_code

200

## **API error alerts (you have to run script outside Colab to enable desktop notifications)**

In [4]:
# from plyer import notification
# from datetime import datetime, date

# data=str(datetime.now())

# def alerta(nome, status, reason, instituicao):
#   notification.notify(
#       title=f'Erro ao acessar a API {nome}.',
#       message=f'Erro {status}, {reason}.\n{data}.',
#       app_name=instituicao,
#       timeout=10 )

# # Alert error API 1:

# if response_taxas_juros.status_code==200:
#   taxas_juros=response_taxas_juros.json() # Posso trocar o nome "data_json" ou tem alguma utilidade? (estava assim nos exemplos em aula: data_json=response_taxas_juros.json()
# else:
#   alerta(nome="taxas_juros", status=response_taxas_juros.status_code, reason=response_taxas_juros.reason, instituicao="Banco Central do Brasil")

# # Alert error API 2:

# if response_expecs.status_code==200:
#       expecs=response_expecs.json()
# else:
#   alerta(nome="expecs", status=response_expecs.status_code, reason=response_expecs.reason, instituicao="Banco Central do Brasil")

# # Alert error API 3:

# if response_ipca.status_code==200:
#       ipca=response_ipca.json()
# else:
#   alerta(nome="ipca", status=response_ipca.status_code, reason=response_ipca.reason, instituicao="IBGE")

# # Alert success APIs:

# def alerta_sucesso():
#   notification.notify(
#       title='Processo finalizado.',
#       message=f'Sucesso ao acessar todas as APIs.\n{data}.',
#       app_name="APIs",
#       timeout=10 )

# if response_taxas_juros.status_code==200 and response_expecs.status_code==200 and response_ipca.status_code==200:
#   alerta_sucesso()


## **Tables preparation**

In [5]:
#TAXAS JUROS
taxas_json=response_taxas_juros.json()
taxas_juros=pd.json_normalize(taxas_json['value'],meta=["Mes", "Modalidade", "Posicao", "InstituicaoFinanceira", "TaxaJurosAoMes", "TaxaJurosAoAno", "cnpj8", "anoMes"]) #JSON para DataFrame
taxas_juros["anoMes"]=pd.to_datetime(taxas_juros["anoMes"], format='%Y-%m')  #transforms object to datetime
taxas_juros=taxas_juros.rename(columns={"anoMes":"Data"})
taxas_juros=taxas_juros.drop(columns=["cnpj8","Mes",'Posicao', 'TaxaJurosAoMes'])
taxas_juros=taxas_juros.set_index("Data")
taxas_juros=taxas_juros[taxas_juros['Modalidade'].str.contains('REGULADAS')==False]
taxas_juros=taxas_juros[taxas_juros['Modalidade'].str.contains('PÓS')==False]
taxas_juros=taxas_juros.groupby(['Data']).mean()
# taxas_juros=taxas_juros.loc[taxas_juros["Data"]>= '2017-01-01']
# taxas_juros = taxas_juros[taxas_juros['Data'] > "2017-01-01"]

#EXPECTATIVAS
expecs_json=response_expecs.json()
expecs=pd.json_normalize(expecs_json['value'],meta=["Indicador", "Data", "Media", "Mediana"]) #JSON to DataFrame
expecs["Data"]=pd.to_datetime(expecs["Data"], format='%Y-%m-%d')  #transforms object to datetime
expecs=expecs.set_index("Data")

#IPCA
ipca_json=response_ipca.json()
ipca=pd.json_normalize(ipca_json, record_path=["resultados",["series"]]) #JSON to DataFrame
ipca=ipca.transpose().drop(["localidade.id","localidade.nivel.id","localidade.nivel.nome","localidade.nome"],axis="index").drop([3,4,5],axis="columns").rename(columns={0:"IPCA - Número-índice",1:"IPCA - Variação mensal",2:"IPCA - Variação acumulada"})
ipca["index"]=ipca["index"].str.slice_replace(0, 6) #removes text from cell values
ipca=ipca.rename(columns={"index":"Data"},inplace=True)
ipca["Data"]=pd.to_datetime(ipca["Data"], format='%Y%m')  #transforms object to datetime
ipca["IPCA - Número-índice"]=pd.to_numeric(ipca["IPCA - Número-índice"])  #transforms object to float
ipca["IPCA - Variação mensal"]=pd.to_numeric(ipca["IPCA - Variação mensal"])  #transforms object to float
ipca["IPCA - Variação acumulada"]=pd.to_numeric(ipca["IPCA - Variação acumulada"])  #transforms object to float


# Filtering of unnecessary lines

# 2- Expecs
expecs=expecs.drop(columns=['Mediana'])
expecs=expecs[expecs['Indicador']=='IPCA']

# 3- IPCA
ipca=ipca.drop(columns=['IPCA - Número-índice', 'IPCA - Variação mensal'])


taxas_juros=taxas_juros.sort_values(by=['Data'])
expecs=expecs.sort_values(by=['Data'])
ipca=ipca.sort_values(by=['Data'])
# pd.set_option('display.max_colwidth', None)
print(taxas_juros.info(), taxas_juros.head(5))
print(expecs.info(), expecs.head(5))
print(ipca.info(), ipca.head(5))

# merge1=taxas_juros.merge(expecs,how="left", on="Data")
# merge1.head(10)

# tabela_final=merge1.merge(ipca,how="left", on="Data")
# tabela_final.info()
# tabela_final.head(10)

<ipython-input-5-ad52fe14f879>:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  taxas_juros=taxas_juros.groupby(['Data']).mean()


KeyError: 'index'

## **Plotting Series**

In [ ]:
#plotting each series
plt.plot(tabela_final["Data"],tabela_final['TaxaJurosAoAno'])
plt.plot(tabela_final["Data"],tabela_final['Media'])
plt.plot(tabela_final["Data"],tabela_final['IPCA - Variação Acumulada'])

#display plot
plt.show()

## **Draft**

In [ ]:
# url_ovos="https://servicodados.ibge.gov.br/api/v3/agregados/6832/periodos/201804|201901|201902|201903|201904|202001|202002|202003|202004|202101|202102|202103|202104|202201|202202|202203|202204|202301|202302|202303/variaveis/29?localidades=N1[all]&classificacao=12716[all]"
# url_pop_estimada="https://servicodados.ibge.gov.br/api/v3/agregados/6579/periodos/2017|2018|2019|2020|2021/variaveis/9324?localidades=N1[all]"
# url_pop_ativa="https://servicodados.ibge.gov.br/api/v3/agregados/3918/periodos/201701|201702|201703|201704|201705|201706|201707|201708|201709|201710|201711|201712|201801|201802|201803|201804|201805|201806|201807|201808|201809|201810|201811|201812|201901|201902|201903|201904|201905|201906|201907|201908|201909|201910|201911|201912|202001|202002|202003|202004|202005|202006|202007|202008|202009|202010|202011|202012|202101|202102|202103|202104|202105|202106|202107|202108|202109|202110|202111|202112|202201|202202|202203|202204|202205|202206|202207|202208|202209|202210|202211|202212|202301|202302|202303|202304|202305|202306|202307|202308|202309|202310/variaveis/4090?localidades=N1[all]&classificacao=12027[all]"
# url_renda="https://servicodados.ibge.gov.br/api/v3/agregados/6390/periodos/201701|201702|201703|201704|201705|201706|201707|201708|201709|201710|201711|201712|201801|201802|201803|201804|201805|201806|201807|201808|201809|201810|201811|201812|201901|201902|201903|201904|201905|201906|201907|201908|201909|201910|201911|201912|202001|202002|202003|202004|202005|202006|202007|202008|202009|202010|202011|202012|202101|202102|202103|202104|202105|202106|202107|202108|202109|202110|202111|202112|202201|202202|202203|202204|202205|202206|202207|202208|202209|202210|202211|202212|202301|202302|202303|202304|202305|202306|202307|202308|202309|202310/variaveis/5929?localidades=N1[all]"

# url_instituicoes="https://olinda.bcb.gov.br/olinda/servico/Informes_ListaTarifasPorInstituicaoFinanceira/versao/v1/odata/ListaInstituicoesDeGrupoConsolidado(CodigoGrupoConsolidado=@CodigoGrupoConsolidado)?@CodigoGrupoConsolidado='03'&$top=10000&$format=json&$select=Cnpj,Nome"
# 61186680
# 00360305
# 01181521
# 00000000
# 00416968
# 60872504
# 90400888
# 60701190
# 33264668
# 60746948
# 61033106
# 31872495
# 30306294
